![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fpresentations&branch=master&subPath=data-science-with-covid-instructor.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/presentations/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Introduction to Data Science with COVID-19 Data

While there are a number of well-designed dashboards and visualization tools for COVID-19 data, such as [Bing](https://bing.com/covid) and [The World Bank](http://datatopics.worldbank.org/universal-health-coverage/coronavirus/), we are going to try building something ourselves in a Jupyter notebook.

This Jupyter notebook uses [COVID-19 statistics from Johns Hopkins University CSSE](https://github.com/CSSEGISandData/COVID-19), you can also see [their dashboard](https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6).

## Licence and Disclaimer

COVID-19 data sets are copyright 2020 [Johns Hopkins University](https://systems.jhu.edu) (available for educational and academic research purposes). The population data is free to use from [Gapminder](https://www.gapminder.org) under a [Creative Commons Attribution license](https://creativecommons.org/licenses/by/4.0/). This notebook also carries a [Creative Commons Attribution license](https://creativecommons.org/licenses/by/4.0/).

This notebook should not be considered medical or policy-making advice. Always follow the directives and orders of your public health authority.

## Getting Started

First, `▶Run` the next cell to import a data set. Once the data set has been downloaded and imported into a [DataFrame](https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe.htm), it will be displayed.

You can change the date, but make sure you use the format `'MM-DD-YYYY'` as they do in the CSSE data set. Files are updated once a day around midnight [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time).

In [2]:
date = '04-07-2020'

import pandas as pd

csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+date+'.csv'
covid_stats = pd.read_csv(csv_url)
covid_stats

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-04-07 23:04:49,34.223334,-82.461707,5,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-04-07 23:04:49,30.295065,-92.414197,82,2,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-04-07 23:04:49,37.767072,-75.632346,11,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-04-07 23:04:49,43.452658,-116.241552,419,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-04-07 23:04:49,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"
...,...,...,...,...,...,...,...,...,...,...,...,...
2852,NaN,NaN,NaN,Vietnam,2020-04-07 23:04:29,14.058324,108.277199,249,0,123,126,Vietnam
2853,NaN,NaN,NaN,West Bank and Gaza,2020-04-07 23:04:29,31.952200,35.233200,261,1,42,218,West Bank and Gaza
2854,NaN,NaN,NaN,Western Sahara,2020-04-07 23:04:29,24.215500,-12.885800,4,0,0,4,",,Western Sahara"
2855,NaN,NaN,NaN,Zambia,2020-04-07 23:04:29,-13.133897,27.849332,39,1,7,31,Zambia


## Data Cleaning

`Run` the next cell to clean up the data. We'll add up values for each country and create a new dataframe.

In [ ]:
# If you prefer specific countries, put a # in front of the next line and remove the six ' marks around the next list
country_list = covid_stats['Country_Region'].unique()
'''
country_list = ['Italy', 'Spain', 'Germany', 'France', 
                'Israel', 'US', 'United Kingdom',
                'Singapore', 'Australia', 'Canada',
                'China', 'Argentina', 'Russia', 'India']
'''

df = pd.DataFrame(columns=['Country', 'Confirmed', 'Recovered', 'Deaths'])

for country in country_list:
    confirmed = covid_stats[covid_stats['Country_Region']==country]['Confirmed'].sum()
    recovered = covid_stats[covid_stats['Country_Region']==country]['Recovered'].sum()
    deaths = covid_stats[covid_stats['Country_Region']==country]['Deaths'].sum()
    data_row = {'Country':country,'Confirmed':confirmed,'Recovered':recovered,'Deaths':deaths}
    df = df.append(data_row, ignore_index=True)

df.sort_values('Confirmed',ascending=False)

## Add World Data

We can also add up all of the values in the data set to get worldwide totals.

In [ ]:
confirmed = covid_stats['Confirmed'].sum()
recovered = covid_stats['Recovered'].sum()
deaths = covid_stats['Deaths'].sum()
world_values = {'Country':'World','Confirmed':confirmed,'Recovered':recovered,'Deaths':deaths}
df = df.append(world_values, ignore_index=True)
df.tail()

## Adding Population Data

We'll use population data from [Gapminder](https://gapminder.org).

In [3]:
pop_sheet_id = '18Ep3s1S0cvlT1ovQG9KdipLEoQ1Ktz5LtTTQpDcWbX0'
pop_gid = '1668956939'
pop_csv_url = 'https://docs.google.com/spreadsheets/d/'+pop_sheet_id+'/export?gid='+pop_gid+'&format=csv'
pop_df = pd.read_csv(pop_csv_url)
current_population = pop_df[pop_df['time']==2019]
current_population

,geo,name,time,population
219,afg,Afghanistan,2019,37209007
520,alb,Albania,2019,2938428
821,dza,Algeria,2019,42679018
1122,and,Andorra,2019,77072
1423,ago,Angola,2019,31787566
...,...,...,...,...
58011,vnm,Vietnam,2019,97429061
58312,yem,Yemen,2019,29579986
58613,zmb,Zambia,2019,18137369
58914,zwe,Zimbabwe,2019,17297495


In [ ]:
cp = current_population.set_index('name')
df.replace('Korea, South','South Korea',regex=True,inplace=True)
df.replace('US','United States',regex=True,inplace=True)
cs = df.set_index('Country')

In [ ]:
new_df = cs.join(cp)
new_df

In [ ]:
new_df.drop(columns=['geo','time'],inplace=True)
new_df.rename(columns={'population':'Population'},inplace=True)
new_df

In [ ]:
new_df = new_df.dropna()
new_df

In [ ]:
new_df['Confirmed Percent'] = new_df['Confirmed']/new_df['Population']*100
new_df

In [ ]:
import cufflinks as cf
cf.go_offline()
new_df.sort_values('Confirmed Percent').tail(20).iplot(kind='bar',y='Confirmed Percent')

## Sorting Data

`Run` the next cell to sort the data by a particular column. The `ascending=False` is optional (the default is `True`), and `.head(16)` shows just the first 16 rows.

In [ ]:
df.sort_values('Confirmed', ascending=False).head(16)

## Selecting Specific Countries

To see a DataFrame of specific countries, edit and run the next cell.

In [ ]:
#df[df['Country']=='Canada']
list_of_countries = ['Canada', 'China', 'Italy']
df[df['Country'].isin(list_of_countries)]

## Graphing Data

We will use the `cufflinks` library to create a graph of our data set.

```python
import cufflinks as cf
cf.go_offline()
df.sort_values('Confirmed').iplot(kind='bar',x='Country',y='Confirmed')
```

Another option:

```python
import cufflinks as cf
cf.go_offline()
df.sort_values('Confirmed',ascending=False).head(20).iplot(kind='bar',x='Country',y='Confirmed',title='COVID Cases')
```

To exclude the `World` row, you can `.drop(184)` (or `.drop('World')` if you've set the index to `'Country'`).

In [ ]:
import cufflinks as cf
cf.go_offline()
df.sort_values('Confirmed').iplot(kind='bar',x='Country',y='Confirmed')

**Hopefully that's an interesting introduction to data science using online COVID-19 data.**

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)